In [1]:
from char_rnn import *
import shutil, os

In [2]:
with open("data/tiny_shakespeare.txt", 'r') as f:
    text = f.read()

print(len(text))
text = text[:1000]
print(text[:10])

1115394
First Citi


In [3]:
# prepare data
train_size = int(0.8 * len(text))
valid_size = int(0.1 * len(text))
test_size = len(text) - train_size - valid_size
train_text = text[:train_size]
valid_text = text[train_size:train_size + valid_size]
test_text = text[train_size + valid_size:]

print(train_size, train_text[:64])
print(valid_size, valid_text[:64])
print(test_size, test_text[:64])

(800, 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAl')
(100, 'articularise their abundance; our\nsufferance is a gain to them L')
(100, 're we become rakes: for the gods know I\nspeak this in hunger for')


In [4]:
# Create vocabulary
unique_chars = list(set(text))
vocab_size = len(unique_chars)
print('vocab size: %d' % vocab_size)
vocab_index_dict = {}
index_vocab_dict = {}

for i, char in enumerate(unique_chars):
    vocab_index_dict[char] = i
    index_vocab_dict[i] = char

vocab size: 46


In [5]:
# Create batch generators
batch_size = 10
n_unrollings = 10
train_batches = BatchGenerator(train_text, batch_size, n_unrollings, vocab_size, 
                               vocab_index_dict, index_vocab_dict)
eval_train_batches = BatchGenerator(train_text, 1, 1, vocab_size,
                                    vocab_index_dict, index_vocab_dict)
valid_batches = BatchGenerator(valid_text, 1, 1, vocab_size,
                               vocab_index_dict, index_vocab_dict)
test_batches = BatchGenerator(test_text, 1, 1, vocab_size,
                              vocab_index_dict, index_vocab_dict)

In [6]:
# Test batch generators
print(batches2string(train_batches.next(), index_vocab_dict))
print(batches2string(valid_batches.next(), index_vocab_dict))

['First Citiz', '\n\nFirst Cit', 'ed. resolve', 'e people.\n\n', ' have corn ', 'be done: aw', 'e are accou', 'ould reliev', 'some, we mi', 'the leannes']
['ar']


In [7]:
# Prepare parameters
params = {'batch_size': batch_size, 'num_unrollings': n_unrollings, 'vocab_size': vocab_size, 
        'hidden_size': 100, 'max_grad_norm': 5.0, 'embedding_size': 50, 
        'num_layers': 1}

In [8]:
# Create graphs
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('char_rnn') as scope:
        with tf.name_scope('training'):
            train_model = CharRNN(is_training=True, **params)
        tf.get_variable_scope().reuse_variables()
        with tf.name_scope('evaluation'):
            valid_model = CharRNN(is_training=False, **params)
        # test_model = CharRNN(is_training=False, **params)

In [9]:
# Training parameters
n_epochs = 10
model_name = 'char_rnn'
saved_model_dir = '/tmp/char_rnn/char_rnn'
init_from_path = '' #'/tmp/saved_char_rnn_model-240'
# init_from_path = saved_path
log_dir = '/tmp/charRNNTrainLog1'

In [10]:
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

In [11]:
print('model size (number of parameters): %s' % train_model.model_size)
with tf.Session(graph=graph) as session:
    writer = tf.train.SummaryWriter(log_dir, session.graph_def)
    if init_from_path:
        train_model.saver.restore(session, init_from_path)
    else:
        tf.initialize_all_variables().run()
        
    for i in range(n_epochs):
        print('\nEpoch %d\n' % i)
        print('training')
        ppl, train_summary_str, global_step = train_model.run_epoch(session, train_size, train_batches, 
                                                 is_training=True, verbose=False)

        writer.add_summary(train_summary_str, global_step)
        saved_path = train_model.saver.save(session, saved_model_dir, 
                                            global_step=train_model.global_step)

        print('model saved in %s\n' % saved_path)
        print('validation')
        valid_ppl, valid_summary_str, _ = valid_model.run_epoch(session, valid_size, valid_batches, 
                                                         is_training=False, verbose=False)
        writer.add_summary(valid_summary_str, global_step)
        writer.flush()
    print('\ntest')
    valid_model.run_epoch(session, test_size, test_batches, 
                         is_training=False, verbose=False)

model size (number of parameters): 67346

Epoch 0

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 67.370, speed: 3254 wps
model saved in /tmp/char_rnn/char_rnn-7

validation
epoch_size: 99
data_size: 100
num_unrollings: 1
batch_size: 1
final ppl: 50.066, speed: 1222 wps

Epoch 1

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 39.808, speed: 4086 wps
model saved in /tmp/char_rnn/char_rnn-14

validation
epoch_size: 99
data_size: 100
num_unrollings: 1
batch_size: 1
final ppl: 27.388, speed: 1060 wps

Epoch 2

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 27.128, speed: 2942 wps
model saved in /tmp/char_rnn/char_rnn-21

validation
epoch_size: 99
data_size: 100
num_unrollings: 1
batch_size: 1
final ppl: 22.532, speed: 908 wps

Epoch 3

training
epoch_size: 7
data_size: 800
num_unrollings: 10
batch_size: 10
final ppl: 25.860, speed: 3578 wps
model saved in /tmp/char_rnn/char_rnn-28

v

In [13]:
# Sampling some sequence that start
start_text = 'Fir'
with tf.Session(graph=graph) as session:
    train_model.saver.restore(session, saved_path)
    print(valid_model.sample_seq(session, 500, start_text, 
                                 vocab_index_dict, index_vocab_dict, max_prob=True))

Fire  iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit iit i


In [18]:
# Compute the loss of some sequence. 
example_text = 'First sissssssssssss'
# example_text = train_text[:20]
example_batches = BatchGenerator(example_text, 1, 1, vocab_size, vocab_index_dict, index_vocab_dict)

In [19]:
with tf.Session(graph=graph) as sess:
    train_model.saver.restore(sess, saved_path)
    print(valid_model.run_epoch(sess, len(try_text), try_batches,
                        is_training=False, verbose=True, debug=False)[0])

epoch_size: 19
data_size: 20
num_unrollings: 1
batch_size: 1
47.4%, step:9, perplexity: 19.067, speed: 454 wps
final ppl: 24.814, speed: 689 wps
24.8144


In [170]:
# ef1 = '/tmp/char_rnn_logs_6/events.out'
# ef2 = '/tmp/char_rnn_logs_3/events.out.tfevents.1457078736.Chens-MacBook-Pro.local'
# for e in tf.train.summary_iterator(ef1):
#     for v in e.summary.value:
#         if v.tag == 'training perplexity':
#             print(v.simple_value)